how to convert anki package to study sauce?


how to sign a file url for uploading to google storage?


In [ ]:
const {Storage} = require('@google-cloud/storage');
const storage = new Storage();

/**
 * HTTP function that generates a signed URL
 * The signed URL can be used to upload files to Google Cloud Storage (GCS)
 *
 * @param {Object} req Cloud Function request context.
 * @param {Object} res Cloud Function response context.
 */
exports.getSignedUrl = (req, res) => {
  if (req.method !== 'POST') {
    // Return a "method not allowed" error
    return res.status(405).end();
  }
  // TODO(developer) check that the user is authorized to upload

  // Get a reference to the destination file in GCS
  const file = storage.bucket(req.body.bucket).file(req.body.filename);

  // Create a temporary upload URL
  const expiresAtMs = Date.now() + 300000; // Link expires in 5 minutes
  const config = {
    action: 'write',
    expires: expiresAtMs,
    contentType: req.body.contentType,
  };

  file.getSignedUrl(config, (err, url) => {
    if (err) {
      console.error(err);
      res.status(500).end();
      return;
    }
    res.send(url);
  });
};



create a copy of study sauce template?


In [ ]:
var uuid = require('uuid/v1');
var importer = require('../Core');
var getSheet = importer.import('get sheet identifier');
var copyFile = importer.import('copy a file on google drive');
var listDrive = importer.import('list google drive files');
var insertPermission = importer.import('insert google drive permissions');

function copyStudy(email) {
    var fileId, newId;
    if(!email) {
        throw new Error('email not specified!');
    }
    return listDrive()
        .then(r => fileId = r.filter(f => f.name === 'Study sauce template')[0].id)
        .then(() => copyFile(fileId, 'Study sauce ' + uuid().substr(0, 5)))
        .then(id => newId = id)
        .then(() => insertPermission(newId, email))
        .then(() => getSheet(newId, null, email))
        .then(() => newId)
}

module.exports = copyStudy;



render study sauce cards page?


In [ ]:
var uuid = require('uuid/v1');
var {Readable} = require('stream');
var importer = require('../Core');
var getTemplates = importer.import('templates google sheet');
var wrapTemplate = importer.import('output google sheet template');
var getTemplateProperties = importer.import('google sheet template properties');

function parseCards(str) {
    return str.split('\n').map(line => {
        var values = line.split(',');
        return {
            type: values[0],
            prompt: values[1],
            answer: values[2],
            'possible-1': values[2],
            'possible-2': values[3],
            'possible-3': values[4],
            'possible-4': values[5]
        }
    })
}

// TODO: make this generic for use with user related packs and state changes?
function renderCards(cards) {
    if(typeof cards === 'string') {
        cards = parseCards(cards)
    }
    // TODO: create a document and render cards from that
    var name = uuid().substr(0, 5) + (new Date()).getTime();
    var properties = {}, templates;
    
    return getTemplates(process.env.DOCID)
        .then(t => {
            templates = t;
            templates['cards'].data.rows = properties['cards-data'];
            templates['cards-cards'] = {template: {rows: [[`{{> cards/cards-cards-link}}`]]}}
            return getTemplateProperties('app', properties, templates);
        })
        .then(() => getTemplateProperties('demo', properties, templates))
        .then(() => getTemplateProperties('cards-cards', properties, templates))
        .then(() => wrapTemplate('cards/' + name, 'cards', properties['cards-cards'], Object.assign(properties, {
            'cards-cards-link': '/cards/' + name,
            'cards-data': (cards || parseCards(properties['demo-text']))
                .map(c => Object.assign(c, {cards: '/cards/' + name}))
        })))
        .then(page => collectTemplateResources(
            `cards/${name}.html`,
            page,
            properties,
            templates,
            process.env.BUCKET))
        .then(resources => (console.log(resources), '/cards/' + name + '.html'))
}

module.exports = renderCards;



authorize app to read profile info?


In [ ]:
var path = require('path');
var fs = require('fs');
var util = require('util');
var cookie = require('cookie');
var {OAuth2Client} = require('google-auth-library');

var importer = require('../Core');
var getDataSheet = importer.import('get google data sheet');
var updateRow = importer.import('update row in google sheets');
var receiveCode = importer.import('receive an authentication code from google');
var extractProfile = receiveCode.extractProfile;

var SCOPES = [
    'https://www.googleapis.com/auth/userinfo.profile',
    'https://www.googleapis.com/auth/userinfo.email'
];
var credentials = require(process.env.SECRETS_PATH);

function authorizeProfile(req, res) {
    var oauth2Client = new OAuth2Client(
        credentials.installed.client_id,
        credentials.installed.client_secret,
        process.env.AUTH_REDIRECT),
        sessId = (req.cookies || {})['__session'];
    var user;

    return getDataSheet(process.env.DOCID, 'User data')
        .then(users => user = users.filter(u => u.session && u.session === sessId)[0])
        .then(() => {
            var token = JSON.parse((user || {}).token || '""');
            if(token) {
                oauth2Client.setCredentials(token)
                return oauth2Client.getAccessToken();
            }
            throw new Error('invalid_token');
        })
        .then(() => extractProfile(oauth2Client))
        .then(() => res.redirect(process.env.DOMAIN + '/' + user.link + '/home/'))
        .catch(up => {
            if(up.message == 'invalid_token') {
                var authUrl = oauth2Client.generateAuthUrl({
                    access_type: 'offline',
                    scope: SCOPES
                });
                res.redirect(authUrl);
                return authUrl;
            } else {
                throw up;
            }
        })
}

module.exports = authorizeProfile;



receive an authentication code from google?

In [ ]:
var uuid = require('uuid/v1');
var uuidSess = require('uuid/v4');

function safeName(name) {
    return name.replace(/[^a-z0-9\-]/ig, '-').substr(0, 40).toLowerCase();
}

function extractProfile(oauth2Client) {
    return util.promisify(oauth2Client.request.bind(oauth2Client))({
            url: 'https://www.googleapis.com/oauth2/v3/userinfo?alt=json'
        })
        .then(r => {
            var profile = r.data;
            var rowData = {
                link: safeName(profile.name) + '-' + uuid().substr(0, 5),
                first: profile.given_name,
                last: profile.family_name,
                email: profile.email,
                token: JSON.stringify(oauth2Client.credentials),
                image: profile.picture || profile.photos[0].value,
                google: profile.id,
                username: profile.name,
                session: uuidSess(),
                date: (new Date).getTime()
            }
            return updateRow(process.env.DOCID, u => u.email === profile.email, rowData, 'User data')
        })
}

function receiveCode(req, res) {
    var oauth2Client = new OAuth2Client(
        credentials.installed.client_id,
        credentials.installed.client_secret,
        process.env.AUTH_REDIRECT)

    var token;
    return util.promisify(oauth2Client.getToken.bind(oauth2Client))(req.query['code'])
        .then(t => token = t)
        .then(() => oauth2Client.setCredentials(token))
        .then(() => extractProfile(oauth2Client))
        .then(profile => {
            res.cookie('__session', profile.session);
            return res.redirect(process.env.DOMAIN + '/home/' + profile.link);
        })
}

module.exports = receiveCode
module.exports.extractProfile = extractProfile



how to create a study sauce user directory?

In [ ]:
var uuid = require('uuid/v1');
var {Readable} = require('stream');
var importer = require('../Core');
var getTemplates = importer.import('templates google sheet');
var wrapTemplate = importer.import('output google sheet template');
var getTemplateProperties = importer.import('google sheet template properties');
var collectTemplateResources = importer.import('collect google sheet resources');


// TODO: make this generic for use with user related packs and state changes?
function renderUser(user) {
    var properties = {}, templates;
    
    return getTemplates(process.env.DOCID)
        .then(t => {
            templates = t;
            return getTemplateProperties('app', properties, templates);
        })
        .then(() => getTemplateProperties('user', properties, templates))
        .then(() => {
            var users = properties['user-data'];
            Object.assign(properties, {
                'base': user + '/',
                'users-users-link': `/${user}/users`
            })
            templates['users-users'] = {template: {rows: [[`{{> users/users-users-link}}`]]}}
        })
        .then(() => getTemplateProperties('users-users', properties, templates))
        .then(() => wrapTemplate(`${user}/users`, 'users', properties['users-users'], properties))
        .then(page => collectTemplateResources(
            `${user}/users`,
            page,
            properties,
            templates,
            process.env.BUCKET))
        .then(resources => (console.log(resources), `/${user}/users`))
}

module.exports = renderUser;



package.json?


In [ ]:
{
    "name": "SheetToWeb",
    "description": "Marketing site functions",
    "license": "UNLICENSED",
    "scripts": {
    },
    "engines": {
        "node": ">= 8",
        "npm": ">= 4"
    },
    "repository": {
        "type": "git",
        "url": "git+https://github.com/megamindbrian/jupytangular.git"
    },
    "dependencies": {
        "@google-cloud/compute": "^0.12.0",
        "@google-cloud/storage": "^2.5.0",
        "googleapis": "^39.2.0",
        "jsdom": "^14.0.0",
        "mustache": "^3.0.1",
        "remarkable": "^1.7.1",
        "cookie": "^0.4.0",
        "cors": "^2.8.5"
    }
}


study-demo.js?


In [ ]:

function createStudyPack(email) {
    jupyter_ops.studysauce.copyStudy({email: email});
    return false;
}

function createCards(cards) {
    jupyter_ops.studysauce.renderCards({cards: cards})
        .then(r => document
              .getElementsByTagName('iframe')[0]
              .setAttribute('src', r))
    return false;
}


